In [1]:
# Integrated UI for Microscopy Stage and Syringe Pumps

# ===== IMPORTS =====
import sys
import os
import base64
import serial
import time
import atexit
import threading
from datetime import datetime
from IPython.display import display, HTML, clear_output, Image
import ipywidgets as widgets
import requests
from syringe_pump_api import SyringePumpController

# ===== CONSTANTS =====
FLOW = 'FLOW'
DIAMETER = 'DIAMETER'
DIRECTION = 'DIRECTION'
STATE = 'STATE'
UNIT = 'UNIT'
GEARBOX = 'GEARBOX'
MICROSTEP = 'MICROSTEP'
ROD = 'ROD'
ENABLE = 'ENABLE'

# ===== HARDWARE COMMUNICATION =====
PI_IP = '192.168.137.3'
#PI_IP = '0.0.0.0'
BASE_URL = f'http://{PI_IP}:5000/api'

def find_pump_port():
    """Try to find the correct COM port for the syringe pump"""
    import serial.tools.list_ports
    ports = list(serial.tools.list_ports.comports())
    for port in ports:
        if 'USB' in port.description.upper() or 'CH340' in port.description.upper():
            return port.device
    return None

def cleanup():
    """Cleanup function to close serial connection on exit"""
    if hasattr(update_pump_settings, 'pump_controller'):
        try:
            update_pump_settings.pump_controller.close()
            print("✅ Cleaned up pump controller connection")
        except Exception as e:
            print(f"⚠️ Error during cleanup: {e}")

# Register cleanup function
atexit.register(cleanup)

# ===== MAIN UI LAYOUT =====
def create_ui():
    
    # Create tab container
    tab = widgets.Tab()
    
    # Create output containers
    status_output = widgets.Output()
    
    # ===== MICROSCOPY TAB =====
    stream_btn = widgets.ToggleButton(description='▶ Start Camera', button_style='success')
    capture_btn = widgets.Button(description='📸 Capture Image')
    save_container = widgets.Output()
    enable_btn = widgets.ToggleButton(description='⏻ Enable Strobe', button_style='success')
    period = widgets.FloatSlider(min=1, max=10000, step=1, value=50, description='Period (µs):')
    width = widgets.FloatSlider(min=0.1, max=1000, step=0.1, value=0.1, description='Width (µs):')
    hold_btn = widgets.ToggleButton(description='🔆 Hold Mode')
    stream_container = widgets.Output()
    captured_container = widgets.Output()
    
    # ===== SYRINGE PUMPS TAB =====
    # Pump selection and controls
    pump_select = widgets.Dropdown(options=['A', 'B', 'C', 'D'], value='A', description='Pump:')
    flow = widgets.FloatSlider(min=0, max=3000, step=1, value=1000, description='Flow:')
    diameter = widgets.FloatSlider(min=1, max=50, step=0.01, value=8.17, description='Diameter (mm):',style={'description_width': 'initial'})
    direction = widgets.ToggleButtons(options=['Infuse', 'Withdraw'], description='Direction:')
    state_btn = widgets.ToggleButton(description='▶ Start Pump', button_style='success')
    unit_select = widgets.Dropdown(options=['UL/MIN', 'UL/HR', 'ML/MIN', 'ML/HR'], value='UL/HR', description='Unit:')
    gearbox = widgets.Dropdown(options=['1:1', '25:1', '100:1'], value='1:1', description='Gearbox:')
    microstep = widgets.Dropdown(options=['1/8', '1/16', '1/32', '1/64'], value='1/16', description='Microstep:')
    threadrod = widgets.Dropdown(options=['1-START', '4-START'], value='1-START', description='Thread Rod:')
    refresh_btn = widgets.Button(description='🔄 Refresh Port', button_style='info')
    pump_status = widgets.Output(layout=widgets.Layout(
    width='300px',  
    height='200px',  
    overflow_y='auto',  
    border='1px solid #ddd',  
    padding='10px',  
    margin='10px 0'  
    ))
    emergency_btn = widgets.Button(
        description='EMERGENCY STOP',
        button_style='warning',
        icon='exclamation-triangle'
    )
    
    def on_refresh_clicked(b):
        if hasattr(update_pump_settings, 'pump_controller'):
            try:
                update_pump_settings.pump_controller.close()
                delattr(update_pump_settings, 'pump_controller')
            except:
                pass
        update_pump_settings()
    
    refresh_btn.on_click(on_refresh_clicked)

    def update_pump_ui_from_config(pump_id):
        """Update UI elements with current pump configuration"""
        if not hasattr(update_pump_settings, 'pump_controller'):
            with pump_status:
                clear_output()
                print("❌ Pump controller not initialized")
            return
            
        try:
            pump = pump_id
            ctrl = update_pump_settings.pump_controller
            
            # Debug: Print available methods
            with pump_status:
                print(f"Available methods: {[m for m in dir(ctrl) if not m.startswith('_')]}")

            # Update UI elements
            flow.value = ctrl.get_flow(pump)
            diameter.value = ctrl.get_diameter(pump)
            
            current_dir = ctrl.get_direction(pump)
            direction.value = 'Infuse' if current_dir == 1 or (isinstance(current_dir, str) and current_dir.upper() == 'INFUSE') else 'Withdraw'
            
            unit_select.value = ctrl.get_unit(pump)
            gearbox.value = ctrl.get_gearbox(pump)
            microstep.value = ctrl.get_microstep(pump)
            threadrod.value = ctrl.get_threadrod(pump)
            state_btn.value = ctrl.get_state(pump)
            state_btn.description = '⏹ Stop Pump' if state_btn.value else '▶ Start Pump'
            
            with pump_status:
                clear_output()
                print(f"✅ Loaded config for pump {pump}")
                
        except Exception as e:
            with pump_status:
                clear_output()
                print(f"❌ Error loading config: {e}")
                import traceback
                traceback.print_exc()

    def on_pump_select(change):
        """Handle pump selection change"""
        if change['name'] == 'value':
            update_pump_ui_from_config(change['new'])

    # Add observer for pump selection
    pump_select.observe(on_pump_select, names='value')

    def create_parameter_handler(param_name, setter_func):
        def handler(change):
            if change['name'] == 'value' and hasattr(update_pump_settings, 'pump_controller'):
                try:
                    pump = pump_select.value
                    value = change['new']
                    getattr(update_pump_settings.pump_controller, setter_func)(pump, value)
                    with pump_status:
                        clear_output()
                        print(f"✅ Updated {param_name} to {value}")
                except Exception as e:
                    with pump_status:
                        clear_output()
                        print(f"❌ Error updating {param_name}: {e}")
        return handler

    # Set up parameter change handlers
    flow.observe(create_parameter_handler('flow', 'set_flow'), 'value')
    diameter.observe(create_parameter_handler('diameter', 'set_diameter'), 'value')
    direction.observe(lambda c: create_parameter_handler('direction', 'set_direction')({
        'name': 'value', 
        'new': 'INFUSE' if c['new'] == 'Infuse' else 'WITHDRAW'
    }), 'value')
    unit_select.observe(create_parameter_handler('unit', 'set_unit'), 'value')
    gearbox.observe(create_parameter_handler('gearbox', 'set_gearbox'), 'value')
    microstep.observe(create_parameter_handler('microstep', 'set_microstep'), 'value')
    threadrod.observe(create_parameter_handler('thread rod', 'set_threadrod'), 'value')

    # State button
    def on_state_click(change):
        if hasattr(update_pump_settings, 'pump_controller'):
            try:
                pump = pump_select.value
                update_pump_settings.pump_controller.set_state(pump, change['new'])
                change['owner'].description = '⏹ Stop Pump' if change['new'] else '▶ Start Pump'
                with pump_status:
                    clear_output()
                    print(f"✅ Pump {'started' if change['new'] else 'stopped'}")
            except Exception as e:
                with pump_status:
                    clear_output()
                    print(f"❌ Error: {e}")

    state_btn.observe(on_state_click, 'value')

    # Emergency stop button
    def emergency_stop(btn):
        """Immediately stop all pumps"""
        if not hasattr(update_pump_settings, 'pump_controller'):
            with pump_status:
                print("❌ No pump controller connected")
            return
        
        try:
            ctrl = update_pump_settings.pump_controller
            # Stop all pumps (A, B, C, D)
            for pump in ['A', 'B', 'C', 'D']:
                try:
                    ctrl.set_state(pump, 'STOP')
                except Exception as e:
                    with pump_status:
                        print(f"⚠️ Error stopping pump {pump}: {e}")
            
            # Update UI
            state_btn.value = False
            state_btn.description = '▶ Start Pump'
            state_btn.button_style = 'success'
            
            with pump_status:
                print("🛑 EMERGENCY STOP: All pumps have been stopped")
        except Exception as e:
            with pump_status:
                print(f"❌ Emergency stop failed: {e}")

    emergency_btn.on_click(emergency_stop)

    def update_strobe(change=None):
        try:
            data = {
                'enable': enable_btn.value,
                'period_ns': int(period.value * 1000),  # Convert to nanoseconds
                'width_ns': int(width.value * 1000),    # Convert to nanoseconds
                'hold': hold_btn.value
            }
            response = requests.post(f'{BASE_URL}/strobe/settings', json=data)
            #response.raise_for_status()
            #with status_output:
            #    print("✅ Strobe settings updated")
        except Exception as e:
            with status_output:
                print(f"❌ Error updating strobe: {e}")

    def toggle_stream(change):
        if change['new']:
            stream_btn.description = '⏹ Stop Camera'
            stream_btn.button_style = 'danger'
            with stream_container:
                clear_output()
                display(HTML(f"""
                    <img src="{BASE_URL}/camera/stream" 
                         style="max-width: 400px; max-height: 300px; border: 2px solid #4CAF50; border-radius: 4px;">
                """))
        else:
            stream_btn.description = '▶ Start Camera'
            stream_btn.button_style = 'success'
            with stream_container:
                clear_output()
                display(HTML('<div style="width:400px; height:300px; border:2px dashed #ccc; display:flex; align-items:center; justify-content:center;">Stream Stopped</div>'))

    def on_strobe_toggle(change):
        if change['name'] == 'value':
            if change['new']:  # Button is being enabled
                enable_btn.description = '⏹ Disable Strobe'
                enable_btn.button_style = 'danger'
            else:  # Button is being disabled
                enable_btn.description = '⏻ Enable Strobe'
                enable_btn.button_style = 'success'
        update_strobe()

    def capture_image(btn):
        try:
            response = requests.get(f'{BASE_URL}/camera/capture')
            response.raise_for_status()
            if response.status_code == 200:
                with captured_container:
                    clear_output()
                    display(Image(data=response.content, width=400))
                with save_container:
                    clear_output()
                    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                    display(HTML(f"""
                        <a href="data:image/jpeg;base64,{base64.b64encode(response.content).decode('utf-8')}"
                           download="capture_{timestamp}.jpg"
                           style="padding: 6px 12px; background-color: #4CAF50; color: white; 
                                  text-decoration: none; border-radius: 4px; display: inline-block;">
                            💾 Save Image
                        </a>
                    """))
                with status_output:
                    print("✅ Image captured! Click 'Save Image' to download.")
        except Exception as e:
            with status_output:
                print(f"❌ Error capturing image: {e}")

    def update_pump_settings(change=None):
        pump = pump_select.value
        try:
            if not hasattr(update_pump_settings, 'pump_controller'):
                port = find_pump_port()
                if port is None:
                    with pump_status:
                        print("❌ Could not find syringe pump. Please check the connection.")
                    return
                try:
                    update_pump_settings.pump_controller = SyringePumpController(port)
                    with pump_status:
                        print(f"✅ Connected to syringe pump on {port}")
                except serial.SerialException as e:
                    with pump_status:
                        print(f"❌ Connect the pump controller again - Serial port error: {e}")
                    return
                except Exception as e:
                    with pump_status:
                        print(f"❌ Failed to initialize pump: {e}")
                    return
            
            try:
                update_pump_settings.pump_controller.set_flow(pump, flow.value)
                update_pump_settings.pump_controller.set_diameter(pump, diameter.value)
                update_pump_settings.pump_controller.set_direction(pump, direction.value.lower())
                update_pump_settings.pump_controller.set_unit(pump, unit_select.value)
                update_pump_settings.pump_controller.set_gearbox(pump, gearbox.value)
                update_pump_settings.pump_controller.set_microstep(pump, microstep.value)
                update_pump_settings.pump_controller.set_threadrod(pump, threadrod.value)
                
                with pump_status:
                    clear_output()
                    print("✅ Pump settings updated")
                    
            except Exception as e:
                with pump_status:
                    print(f"❌ Failed to update pump settings: {e}")

            if not hasattr(update_pump_settings, 'initialized'):
                # Load initial configuration for the first pump
                update_pump_ui_from_config(pump_select.value)
                update_pump_settings.initialized = True

        except Exception as e:
            with pump_status:
                print(f"❌ Unexpected error: {e}")
    
    def toggle_pump(change):
        if not hasattr(update_pump_settings, 'pump_controller'):
            with pump_status:
                print("❌ Please refresh the port first")
            state_btn.value = False
            return
            
        pump = pump_select.value
        try:
            if change['new']:  # Start pump
                update_pump_settings.pump_controller.set_state(pump, 'RUN')
                state_btn.description = '⏹ Stop Pump'
                state_btn.button_style = 'danger'
                with pump_status:
                    print(f"▶ Pump {pump} started at {flow.value} {unit_select.value}")
            else:  # Stop pump
                update_pump_settings.pump_controller.set_state(pump, 'STOP')
                state_btn.description = '▶ Start Pump'
                state_btn.button_style = 'success'
                with pump_status:
                    print(f"⏹ Pump {pump} stopped")
        except Exception as e:
            with pump_status:
                print(f"❌ Error controlling pump: {e}")
            state_btn.value = False
    
    # ===== LAYOUT =====
    camera_controls = widgets.HBox([stream_btn, capture_btn, save_container])
    strobe_controls = widgets.VBox([
        widgets.HBox([enable_btn, hold_btn]),
        period,
        width])
    image_display = widgets.HBox([
        widgets.VBox([widgets.Label('Live Stream'), stream_container]),
        widgets.VBox([widgets.Label('Captured Image'), captured_container])
    ])
    
    microscopy_tab = widgets.VBox([
        widgets.HTML("<h3>Camera Controls</h3>"),
        camera_controls,
        widgets.HTML("<h3>Strobe Controls</h3>"),
        strobe_controls,
        widgets.HTML("<hr>"),
        image_display
    ])

    pump_basic = widgets.VBox([
        widgets.HTML("<h3>Basic Controls</h3>"),
        pump_select,
        unit_select,
        flow,
        diameter,
        direction,
        widgets.Label(''),   # Empty label for spacing
        widgets.HBox([  # Put both buttons in an HBox
            state_btn,
            emergency_btn
        ], layout=widgets.Layout(justify_content='space-between'))   
    ], layout=widgets.Layout(margin='0 20px 0 0'))  # Right margin for spacing
    
    pump_advanced = widgets.VBox([
        widgets.HTML("<h3>Advanced Settings</h3>"),
        gearbox,
        microstep,
        threadrod
    ], layout=widgets.Layout(margin='0 20px 0 0'))

    pump_comm = widgets.VBox([
        widgets.HTML("<h3>Status</h3>"),
        refresh_btn,
        pump_status
    ], layout=widgets.Layout(
        min_width='350px',  # Ensure enough width for the status box
        margin='0 0 0 20px'  # Left margin to separate from other sections
    ))
    
    # Create a horizontal layout with spacing
    pumps_tab = widgets.HBox([
        pump_basic,
        pump_advanced,
        pump_comm
    ])
    
    # Set up tabs
    tab.children = [microscopy_tab, pumps_tab]
    tab.titles = ['Microscope', 'Syringe Pumps']
    
    # Display UI
    display(tab)
    display(status_output)
    
    # Initialize containers
    with stream_container:
        display(HTML('<div style="width:400px; height:300px; border:2px dashed #ccc; display:flex; align-items:center; justify-content:center;">Stream Stopped</div>'))
    with captured_container:
        display(HTML('<div style="width:400px; height:300px; border:2px dashed #ccc; display:flex; align-items:center; justify-content:center;">No Image Captured</div>'))
    
    # ===== EVENT HANDLERS =====
    #enable_btn.observe(update_strobe, 'value')
    enable_btn.observe(on_strobe_toggle, 'value')
    period.observe(update_strobe, 'value')
    width.observe(update_strobe, 'value')
    hold_btn.observe(update_strobe, 'value')
    stream_btn.observe(toggle_stream, 'value')
    capture_btn.on_click(capture_image)
    state_btn.observe(toggle_pump, 'value')

In [2]:
# Start the UI
create_ui()

Output()

In [ ]:
# Integration with LabThings Retro

import matplotlib.pyplot as plt
import numpy as np
import time

#LabThings import
import atexit
from labthings_client.discovery import ThingBrowser

browser = ThingBrowser().open()
atexit.register(browser.close)
time.sleep(10)

#There are two devices, one with ID labThingsRetroRed and one with ID labThingsRetroBlue
if browser.things[0].thing_description.get('id') == 'labThingsRetroRed':
    pumpRed = browser.things[0]
    #pumpBlue = browser.things[1]
else:
    pumpBlue = browser.things[0]
    #pumpRed = browser.things[1]

pumpBlue.properties.pumpRng.set('2') #ml/mn
pumpBlue.properties.pumpDia.set(8.17)
pumpBlue.properties.pumpRat.set(2000)

In [ ]:
pumpBlue.actions.pumpInfuse()
pumping = 'blue'
print('Pumping blue')

In [ ]:
pumpBlue.actions.pumpStop()